In [ ]:
!nvidia-smi

# environment

In [ ]:
!pip install gdown --quiet

### code

In [ ]:
%%bash
File_id="1eSGdjEsfusyZkXwsENAeZlCQE7J7dDjB"
File_name="benchmark"

gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"
unzip -q "${File_name}.zip" -d "${File_name}"
# rm -f "${File_name}.zip"

In [ ]:
%cd benchmark/

### package

In [ ]:
!pip install -r requirements.txt --quiet # --q 讓他安靜

### data

In [ ]:
%%bash
File_id="1ea8mw6kYs6UbhMgSeVPR9tl-Agio2Ln9"
gdown "https://drive.google.com/uc?id=${File_id}&confirm=t"

In [ ]:
%%bash
folder="data/train/gm12878_ctcf"

mkdir -p $folder
unzip -q "gm12878_ctcf.zip" -d $folder

In [ ]:
!ls

In [ ]:
# !rm -r output

### run

In [ ]:
%%bash
#!/bin/bash
name="gm12878_ctcf_2024.04.29_cnn-pairs_hyper-test"

# Define configuration variables
config="conf/base-CNN_pairs.yaml"
input_folder="data/train/gm12878_ctcf"
out_folder_base="output/${name}"
input_format="onehot"

learning_rates=(0.001)
hidden_sizes=(200 300)
extractor_hidden_size=(100 200)
hidden_layer=(1 2)

# bp_range=(1000 2000 3000)
bp_range=(3000)

# Calculate total tasks
total_tasks=$((
    ${#learning_rates[@]} * ${#hidden_sizes[@]} *
    ${#hidden_layer[@]} * ${#bp_range[@]} * ${#extractor_hidden_size[@]}
))
completed_tasks=1

# Define hyperparameters (using loops)
for lr in ${learning_rates[@]}; do
    for hs in ${hidden_sizes[@]}; do
        for hl in ${hidden_layer[@]}; do
            for bp in ${bp_range[@]}; do
                for ehs in ${extractor_hidden_size[@]}; do
                  # Construct output folder name with experiment details
                  out_folder="${out_folder_base}/Lr${lr}_CHs${ehs}_Hs${hs}_Hl${hl}_Bp${bp}"
                  
                  echo "[$(($completed_tasks))/$total_tasks] Running experiment: $out_folder"

                  # Train with the specified hyperparameters
                  python train.py \
                    --config "$config" \
                    --input "${input_folder}/${bp}bp.50ms.${input_format}/data.h5" \
                    --output_folder "$out_folder" \
                    --device gpu \
                    --eval_freq 1 \
                    --pin_memory_train True \
                    --data.anchor_size $bp \
                    --train.learning_rate $lr \
                    --model.classifier_hidden_size $hs \
                    --model.extractor_hidden_size $ehs \
                    --model.classifier_hidden_layer_n $hl

                  # 檢查執行結果
                  if [ $? -ne 0 ]; then
                    echo "Error: Training failed with batch size $batch_size"
                    exit 1
                  fi

                  ((completed_tasks++))
                done
            done
        done
    done
done

# Sort experiment results
# python code/helper_scripts/sort_exp_result.py --folder "$out_folder_base"

In [ ]:
%%bash
name="gm12878_ctcf_2024.04.29_cnn-pairs_hyper-test"
folder="output"

zip -r "${folder}/${name}.zip" "${folder}/${name}"

In [ ]:
# 一定要在這路徑底下才行
import os
os.chdir('/kaggle/working')

from IPython.display import FileLink
FileLink('benchmark/output/gm12878_ctcf_2024.04.29_cnn-pairs_hyper-test.zip')